In [1]:
from collections import Counter
from functools import reduce
from datetime import datetime

import numpy as np

In [2]:
with open('input/12-23-input', 'r') as f:
    data = [x.strip() for x in f]

data = [
'....#..',
'..###.#',
'#...#.#',
'.#...##',
'#.###..',
'##.#.##',
'.#..#..'
]

In [3]:
def pad(M):
    #  Pads a numpy array with a ring of 0s, i.e. adds a new top row, new bottom row, 
    #  new first column, and new last column of zeros.
    dim = M.shape
    return np.r_['0,2', [0]*(dim[1]+2), np.c_[np.zeros(dim[0]), M, np.zeros(dim[0])], [0]*(dim[1]+2)]

In [4]:
def trim(M):
    #  Trims the numpy array of outer rows/columns of zeros
    result = M.copy()
    while np.all(result[0, :] == 0):
            result = result[1:, :]
    while np.all(result[-1, :] == 0):
        result = result[:-1, :]
    while np.all(result[:, 0] == 0):
        result = result[:, 1:]
    while np.all(result[:, -1] == 0):
        result = result[:, :-1]
    return result

###  Part 1

In [5]:
positions = np.array([ [0 if x == '.' else 1 for x in y] for y in data])
directions = ['N', 'S', 'W', 'E']

for _ in range(10):
    positions = pad(positions)
    elves = zip(np.nonzero(positions)[0].tolist(), np.nonzero(positions)[1].tolist())
    #  Track the starting position and the desired end position for each elf
    goal = {}
    for row, col in elves:
        window = positions[row-1:row+2, col-1:col+2].copy()
        #  Initially assume that the elf doesn't want to move
        goal[(row, col)] = (row, col)
        if window.sum() == 1:  #  no neighbors, so will stay in the same place
            pass
        else:  #  elf wants to move
            done = False
            for d in directions:
                if d == 'N' and not done:
                    north = window[0, :].sum()
                    if north == 0:
                        done = True
                        goal[(row, col)] = (row-1, col)
                if d == 'S' and not done:
                    south = window[2, :].sum()
                    if south == 0:
                        done = True
                        goal[(row, col)] = (row+1, col)
                if d == 'W' and not done:
                    west = window[:, 0].sum()
                    if west == 0:
                        done = True
                        goal[(row, col)] = (row, col-1)
                if d == 'E' and not done:
                    east = window[:, 2].sum()
                    if east == 0:
                        done = True
                        goal[(row, col)] = (row, col+1)
                        
    wants = [k for k, v in Counter(goal.values()).items() if v == 1]
    valid_moves = { x: y for x,y in goal.items() if y in wants and x != y }
    for k, v in valid_moves.items():
        positions[k] = 0
        positions[v] = 1             
    
    #  Update list of considered directions
    directions = directions[1:] + [directions[0]]

In [6]:
positions = trim(positions)

int(reduce(lambda x,y: x*y, positions.shape) - positions.sum())

3757

### Part 2

In [7]:
positions = np.array([ [0 if x == '.' else 1 for x in y] for y in data])
directions = ['N', 'S', 'W', 'E']

rounds = 0

while True:
    rounds += 1
    positions = pad(positions)
    elves = zip(np.nonzero(positions)[0].tolist(), np.nonzero(positions)[1].tolist())
    #  Track the starting position and the desired end position for each elf
    goal = {}
    for row, col in elves:
        window = positions[row-1:row+2, col-1:col+2].copy()
        #  Initially assume that the elf doesn't want to move
        goal[(row, col)] = (row, col)
        if window.sum() == 1:  #  no neighbors, so will stay in the same place
            pass
        else:  #  elf wants to move
            done = False
            for d in directions:
                if d == 'N' and not done:
                    north = window[0, :].sum()
                    if north == 0:
                        done = True
                        goal[(row, col)] = (row-1, col)
                if d == 'S' and not done:
                    south = window[2, :].sum()
                    if south == 0:
                        done = True
                        goal[(row, col)] = (row+1, col)
                if d == 'W' and not done:
                    west = window[:, 0].sum()
                    if west == 0:
                        done = True
                        goal[(row, col)] = (row, col-1)
                if d == 'E' and not done:
                    east = window[:, 2].sum()
                    if east == 0:
                        done = True
                        goal[(row, col)] = (row, col+1)
    
    wants = [k for k, v in Counter(goal.values()).items() if v == 1]
    valid_moves = { x: y for x,y in goal.items() if y in wants and x != y}
    if len(valid_moves) == 0:
        print(f'Done after {rounds} rounds!')
        break
    for k, v in valid_moves.items():
        positions[k] = 0
        positions[v] = 1             
    
    #  Update list of considered directions
    directions = directions[1:] + [directions[0]]
    if rounds % 10 == 0:
        positions = trim(positions)
    if rounds % 1000 == 0:
        print(f'{datetime.now()}  {rounds}')

Done after 918 rounds!
